In [2]:
# 사각형 내 상가분석
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import pandas as pd
import json
skey = '2WKhofO%2FP%2Fzyhb%2FYYUq6fVtO08ZFBBiJ%2B7abwe9sV7Y%2BP9sWIADFvAVKjMJc6qln2Kabc6cgEs4anTIddxB7Tg%3D%3D'
res_id = 'store'

def baro_api(skey, res_id, encode):
    stype = 'json'
    url = 'http://apis.data.go.kr/B553077/api/open/sdsc2/baroApi'
    params = f'?ServiceKey={skey}&type={stype}&resId={res_id}&' + encode   
    query = url + params
    
    response = Request(query)
    response.get_method = lambda: 'GET'
    response_body = urlopen(response).read()
    
    response_dict = json.loads(response_body.decode('utf-8'))
    items = response_dict['body']['items']

    df = pd.DataFrame(items)
    return df    

In [14]:
# 역 주변 350m반경 내의 상가정보
# 역 좌표에 경도, 위도 대비 350m만큼의 사각좌표 구하기
def xy(x,y):
    max_x = x + 0.00350
    max_y = y + 0.00350
    min_x = x - 0.00350    
    min_y = y - 0.00350        
    return (min_x, min_y, max_x, max_y)
# xy좌표 값 : 이대역 좌표
min_x, min_y, max_x, max_y = xy(126.945857, 37.556761)

import numpy as np
#좌표로 계산된 지역, 구역 나누기
print(min_x, min_y, max_x, max_y )
li_x = np.linspace(min_x,max_x,20)
li_y = np.linspace(min_y,max_y,20)
#좌표 API 내에 넣고 데이터 프레임 만들기
df_s = np.NaN
for i in range(len(li_x)-1):
    x0 = li_x[i]
    x1 = li_x[i+1]
    for j in range(len(li_y)-1):
        y0 = li_y[j]
        y1 = li_y[j+1]
        
        # #이대역 37.556761, 126.945857
        encode = urlencode({quote_plus('catId'):'rectangle'
                            , quote_plus('minx'):x0, quote_plus('miny'):y0
                            , quote_plus('maxx'):x1, quote_plus('maxy'):y1})

        df_t = baro_api(skey, res_id, encode)
        #print(i, j, type(df_t), len(df_t))

        if type(df_t) == type(pd.DataFrame()) and len(df_t) > 0:        
            if type(df_s) == type(pd.DataFrame()) and len(df_s) > 0:
                #df_s = pd.concat(df_s,df_t, ignore_index=True)
                df_s = pd.concat([df_s, df_t], axis = 0, ignore_index=True)
            else:
                df_s = df_t    

df_s

126.942357 37.553261 126.949357 37.560261000000004


,bizesId,bizesNm,brchNm,indsLclsCd,indsLclsNm,indsMclsCd,indsMclsNm,indsSclsCd,indsSclsNm,ksicCd,...,bldMngNo,bldNm,rdnmAdr,oldZipcd,newZipcd,dongNo,flrNo,hoNo,lon,lat
0,22414172,브레드칸,,N,관광/여가/오락,N03,연극/영화/극장,N03A06,연극/음악/예술관련기타,,...,1144011000100040010020803,,서울특별시 마포구 신촌로24안길 16-3,121806,04103,,,,126.942434,37.555489
1,12098251,머리쟁이,,F,생활서비스,F01,이/미용/건강,F01A01,여성미용실,S96112,...,1144011000100010067026481,,서울특별시 마포구 신촌로 154,121806,04104,,2,,126.942637,37.556347
2,12620998,에이치에비뉴호텔이대점,이대점,O,숙박,O01,호텔/콘도,O01A01,호텔/콘도,,...,1144011000100010074006855,대성목재,서울특별시 마포구 신촌로 152,121806,04104,,1,,126.942560,37.556238
3,21954392,로뎀원룸리빙텔,,R,학문/교육,R10,도서관/독서실,R10A02,고시원,R90212,...,1144011000100010067026481,,서울특별시 마포구 신촌로 154,121806,04104,,,,126.942637,37.556347
4,11980901,팔색,,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1141011200101040036000001,신촌자이엘라,서울특별시 서대문구 신촌로 149,120809,03780,,1,,126.942483,37.556859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,11755132,파파존스피자,이대아현점,Q,음식,Q06,양식,Q06A01,정통양식/경양식,I56114,...,1141011000102510050026947,,서울특별시 서대문구 신촌로 207,120822,03767,,2,,126.949133,37.557207
953,12452641,파파존스이대아현점,이대아현점,Q,음식,Q07,패스트푸드,Q07A01,피자전문,I56192,...,1141011000102510050026947,,서울특별시 서대문구 신촌로 207,120822,03767,,1,,126.949133,37.557207
954,21946384,로신이,,D,소매,D06,가방/신발/액세서리,D06A02,구두제화/캐주얼화,G47420,...,1141011000102510050026947,,서울특별시 서대문구 신촌로 207,120822,03767,,1,,126.949133,37.557207
955,26159771,뮤즈웨딩,,F,생활서비스,F17,예식/의례/관혼상제,F17A01,예식장/예식서비스,S96991,...,1141011000102510050026947,,서울특별시 서대문구 신촌로 207,120822,03767,,1,,126.949133,37.557207


In [11]:
# 데이터프레임 내에 중복값 확인 및 처리
print(df_s.duplicated().sum())

0


In [13]:
df_s.drop_duplicates(inplace = True)
# 중복값 재확인
df_s.duplicated().sum()

0